In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Generar de nuevo el diccionario teniendo en cuenta la diferencia
# entre uint e int
dict_dtypes = {
    'IsBeta': np.int8,                                                              
    'RtpStateBitfield': np.int8,                                                    
    'IsSxsPassiveMode': np.int8,                                                    
    'DefaultBrowsersIdentifier': np.int16,                                          
    'AVProductStatesIdentifier': np.int32,                                          
    'AVProductsInstalled': np.int8,                                                 
    'AVProductsEnabled': np.int8,
    'HasTpm': np.int8,                                                              
    'CountryIdentifier': np.int16,
    'CityIdentifier': np.int32,                                                     
    'OrganizationIdentifier': np.int8,
    'GeoNameIdentifier': np.int16,                                                  
    'LocaleEnglishNameIdentifier': np.int16,                                        
    'OsBuild': np.int16,                                                            
    'OsSuite': np.int16,                                                            
    'IsProtected': np.int8,                                                         
    'AutoSampleOptIn': np.int8,                                                     
    'SMode': np.int8,                                                               
    'IeVerIdentifier': np.int16,                                                    
    'Firewall': np.int8,                                                            
    'UacLuaenable': np.int32,                                                       
    'Census_OEMNameIdentifier': np.int16,                                           
    'Census_OEMModelIdentifier': np.int32,                                          
    'Census_ProcessorCoreCount': np.int16,
    'Census_ProcessorManufacturerIdentifier': np.int8,                              
    'Census_ProcessorModelIdentifier': np.int16,
    'Census_PrimaryDiskTotalCapacity': np.int32,                                    
    'Census_SystemVolumeTotalCapacity': np.int32,                                   
    'Census_HasOpticalDiskDrive': np.int8,                                          
    'Census_TotalPhysicalRAM': np.int32,                                            
    'Census_InternalPrimaryDiagonalDisplaySizeInInches': np.float16,                  
    'Census_InternalPrimaryDisplayResolutionHorizontal': np.int16,                  
    'Census_InternalPrimaryDisplayResolutionVertical': np.int16,
    'Census_InternalBatteryNumberOfCharges': np.int32,
    'Census_OSBuildNumber': np.int16,                                               
    'Census_OSBuildRevision': np.int32,                                             
    'Census_OSInstallLanguageIdentifier': np.int8,                                  
    'Census_OSUILocaleIdentifier': np.int16,                                        
    'Census_IsPortableOperatingSystem': np.int8,                                    
    'Census_IsFlightingInternal': np.int8,                                          
    'Census_IsFlightsDisabled': np.int8,                                            
    'Census_ThresholdOptIn': np.int8,                                               
    'Census_FirmwareManufacturerIdentifier': np.int16,
    'Census_FirmwareVersionIdentifier': np.int32,                                   
    'Census_IsSecureBootEnabled': np.int8,                                          
    'Census_IsWIMBootEnabled': np.int8,                                             
    'Census_IsVirtualDevice': np.int8,
    'Census_IsTouchEnabled': np.int8,                                               
    'Census_IsPenCapable': np.int8,                                                 
    'Census_IsAlwaysOnAlwaysConnectedCapable': np.int8,                             
    'Wdft_IsGamer': np.int8,
    'Wdft_RegionIdentifier': np.int8,
    'HasDetections': np.int8,
    'Census_InternalBatteryType_informed': np.int8,                                 
    'ProductName_index': np.int8,                                                   
    'Platform_index': np.int8,
    'Processor_index': np.int8,
    'OsPlatformSubRelease_index': np.int8,                                          
    'SkuEdition_index': np.int8,
    'PuaMode_index': np.int8,
    'SmartScreen_index': np.int8,
    'Census_MDC2FormFactor_index': np.int8,                                         
    'Census_DeviceFamily_index': np.int8,                                           
    'Census_ProcessorClass_index': np.int8,                                         
    'Census_PrimaryDiskTypeName_index': np.int8,
    'Census_ChassisTypeName_index': np.int8,
    'Census_PowerPlatformRoleName_index': np.int8,
    'Census_InternalBatteryType_index': np.int8,                                    
    'Census_OSArchitecture_index': np.int8,                                         
    'Census_OSBranch_index': np.int8,                                               
    'Census_OSEdition_index': np.int8,                                              
    'Census_OSSkuName_index': np.int8,                                              
    'Census_OSInstallTypeName_index': np.int8,                                      
    'Census_OSWUAutoUpdateOptionsName_index': np.int8,
    'Census_GenuineStateName_index': np.int8,                                       
    'Census_ActivationChannel_index': np.int8,
    'Census_FlightRing_index': np.int8,
    'OSBuild_diff': np.int8,
    'AvSigVersion_diff': np.int16,                                                  
    'OSBuild_fulldiff': np.int8,                                                    
    'AvSigVersion_fulldiff': np.int16,                                              
    'OsBuildLab_difftotal': np.int16,                                               
    'DateAvSigVersion_difftotal': np.int16,                                         
    'DateAvSigVersion_fulldifftotal': np.int16,
    'OsBuildLab_fulldifftotal': np.int16,
    'DateAvSigVersion_ratio': np.float16,                                              
    'OsBuildLab_ratio': np.float16,                                                    
    'DateAvSigVersion_fullratio': np.float16,
    'OsBuildLab_fullratio': np.float16,                                                
    'OsBuildLab_dayOfWeek': np.int8,                                                
    'AvSigVersion_dayOfWeek': np.int8, 
}

In [4]:
path = '../data/train_final_1'

In [4]:
# import os
# import pandas as pd 
# from multiprocessing import Pool

# # wrap your csv importer in a function that can be mapped
# def read_csv(filename):
#     'converts a filename to a pandas dataframe'
#     return pd.read_csv(filename)


# def readCsv(processes=8, path=''):
#     file_list = glob.glob(path + "/*.csv")
#     # set up your pool
#     pool = Pool(processes=processes) # or whatever your hardware can support

#     # get a list of file names
#     files = os.listdir(path)

#     # have your pool map the file names to dataframes
#     df_list = pool.map(read_csv, file_list)

#     # reduce the list of dataframes to a single dataframe
#     combined_df = pd.concat(df_list, ignore_index=True)
    
#     return combined_df


In [6]:
# a = readCsv(path = path)

In [7]:
# import blaze
# import pandas as pd

# csv_data = blaze.Data('input.csv')
# query = csv_data[csv_data['a'] > csv_data['b']]
# df = pd.DataFrame.from_records(query, columns=query.fields)

In [5]:
allFiles = glob.glob(path + "/*.csv")

list_ = []
n = 0

for file_ in allFiles:
#     mask = file_['Platform_index'] != 0
    df = pd.read_csv(file_, dtype=dict_dtypes)
    list_.append(df)
    if n%10==0:
        print(n)
    n+=1

train = pd.concat(list_, axis = 0, ignore_index = True)

0
10


In [6]:
train.shape
# (8921483, 100)

(8921483, 92)

In [7]:
train.memory_usage(deep=True).sum()/1024**2

2007.9327278137207

In [8]:
train.head()

,MachineIdentifier,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,OsBuildLab_difftotal,DateAvSigVersion_difftotal,DateAvSigVersion_fulldifftotal,OsBuildLab_fulldifftotal,DateAvSigVersion_ratio,OsBuildLab_ratio,DateAvSigVersion_fullratio,OsBuildLab_fullratio,OsBuildLab_dayOfWeek,AvSigVersion_dayOfWeek
0,00001b924fcc6922321cfadbafd8a91a,0,7,0,0,47238,2,1,1,164,...,224,95,95,224,0.0,0.0,0.0,0.0,2,3
1,000037f84e21c83328ba6963cdac497b,0,7,0,0,7945,2,1,1,12,...,224,91,91,224,0.0,0.0,0.0,0.0,2,7
2,000079a261a28c51c4ae8393c7fd97e8,0,7,0,0,53447,1,1,1,195,...,477,142,142,477,0.0,0.0,0.0,0.0,1,5
3,000079e9300ce84a22df348cd25e1d60,0,7,0,0,49480,2,1,1,214,...,236,87,87,236,0.0,0.0,0.0,0.0,4,4
4,0000a313e380907701398b3c61f58848,0,7,0,0,7945,2,1,1,141,...,202,89,89,202,0.0,0.0,0.0,0.0,3,2


In [24]:
# Platform_index
train_1 = train[train['Platform_index']==1]

In [25]:
train_1.shape

(194508, 100)

In [78]:
# train_mini = train.iloc[:700000,:]

In [14]:
sel_cols = [c for c in train.columns if c not in ['MachineIdentifier', 'HasDetections']]

In [27]:
X = train_1.loc[:, sel_cols]
y = train_1.loc[:,'HasDetections']

{'bagging_fraction': 0.8, 'bagging_freq': 1, 'boosting_type': 'gbdt', 'feature_fraction': 0.6, 'lambda_l1': 0.2, 'lambda_l2': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 19, 'metric': 'binary_error', 'min_data_in_leaf': 30, 'num_leaves': 43, 'objective': 'binary'}

In [81]:
params = {
    'objective' :['binary'],
    'learning_rate' : [0.1],
    'num_leaves' : [43],
    'feature_fraction': [0.6], 
    'bagging_fraction': [0.8], 
    'bagging_freq':[1],
    'boosting_type' : ['gbdt'],
    'metric': ['binary_error'],
    'max_depth': [19],
    'min_data_in_leaf': [30],
    'max_delta_step': [0],
    'lambda_l1': [0.2],
    'lambda_l2': [0]
}

kFolds = 7

In [82]:
mdl_lgb = lgb.LGBMClassifier(n_jobs=1, task='refit')

In [83]:
lgb_model = GridSearchCV(
    estimator=mdl_lgb,
    param_grid=params,
    n_jobs=-1,
    cv=kFolds,
    verbose=1,
    scoring='roc_auc'
)

In [63]:
# lgb_model.fit(X, y)

In [64]:
# lgb_model.best_score_

In [65]:
# mdl_lgb.fit(X, y)

Entrenamiento del modelo para los distintos Windows disponibles

In [67]:
for i in [3, 2, 1]:
    print('Entrenando modelo para Platform_index = {}'.format(i))
    train_1 = train[train['Platform_index']==i]
    print("Nº de casos en el train = {}".format(train_1.shape[0]))
    X = train_1.loc[:, sel_cols]
    y = train_1.loc[:,'HasDetections']
    lgb_model.fit(X, y)
    print("Mejores parametros:\n{}".format(lgb_model.best_params_))
#     print("Score por split en CV")
#     for i in range(kFolds):
#         te = lgb_model.cv_results_['split{}_test_score'.format(i)]
#         tr = lgb_model.cv_results_['split{}_train_score'.format(i)]
#         print('Split nº {}: train AUC = {} | test AUC = {}'.format(i, round(tr[0],3), round(te[0],3)))
    be = lgb_model.best_score_
    print('Mejor score para Platform_index = {} es: AUC = {}\n'.format(i, round(be, 3)))

Entrenando modelo para Platform_index = 3
Nº de casos en el train = 14371
Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 192 out of 192 | elapsed:   28.6s finished


Mejores parametros:
{'bagging_fraction': 0.8, 'bagging_freq': 1, 'boosting_type': 'gbdt', 'feature_fraction': 0.6, 'lambda_l1': 0, 'lambda_l2': 0.2, 'learning_rate': 0.05, 'max_delta_step': 0, 'max_depth': 19, 'metric': 'binary_error', 'min_data_in_leaf': 30, 'num_leaves': 43, 'objective': 'binary'}
Mejor score para Platform_index = 3 es: AUC = 0.766

Entrenando modelo para Platform_index = 2
Nº de casos en el train = 93889
Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   35.0s
[Parallel(n_jobs=-1)]: Done 192 out of 192 | elapsed:  2.5min finished


Mejores parametros:
{'bagging_fraction': 0.8, 'bagging_freq': 1, 'boosting_type': 'gbdt', 'feature_fraction': 0.6, 'lambda_l1': 0.2, 'lambda_l2': 0, 'learning_rate': 0.1, 'max_delta_step': 0.2, 'max_depth': 19, 'metric': 'binary_error', 'min_data_in_leaf': 30, 'num_leaves': 43, 'objective': 'binary'}
Mejor score para Platform_index = 2 es: AUC = 0.685

Entrenando modelo para Platform_index = 1
Nº de casos en el train = 194508
Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 192 out of 192 | elapsed:  5.4min finished


Mejores parametros:
{'bagging_fraction': 0.8, 'bagging_freq': 1, 'boosting_type': 'gbdt', 'feature_fraction': 0.6, 'lambda_l1': 0.2, 'lambda_l2': 0.2, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 15, 'metric': 'binary_error', 'min_data_in_leaf': 30, 'num_leaves': 43, 'objective': 'binary'}
Mejor score para Platform_index = 1 es: AUC = 0.713



In [71]:
params_1 = {'bagging_fraction': 0.8, 'bagging_freq': 1, 'boosting_type': 'gbdt', 'feature_fraction': 0.6, 'lambda_l1': 0.2, 'lambda_l2': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 19, 'metric': 'binary_error', 'min_data_in_leaf': 30, 'num_leaves': 43, 'objective': 'binary'}

In [84]:
print('Entrenando modelo para Platform_index = {}'.format(i))
train_1 = train[train['Platform_index']==0].sample(500000)
print("Nº de casos en el train = {}".format(train_1.shape[0]))
X = train_1.loc[:, sel_cols]
y = train_1.loc[:,'HasDetections']
lgb_model.fit(X, y)
print("Mejores parametros:\n{}".format(lgb_model.best_params_))
be = lgb_model.best_score_
print('Mejor score para Platform_index = {} es: AUC = {}\n'.format(i, round(be, 3)))

Entrenando modelo para Platform_index = 1
Nº de casos en el train = 500000
Fitting 7 folds for each of 1 candidates, totalling 7 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:   35.1s finished


Mejores parametros:
{'bagging_fraction': 0.8, 'bagging_freq': 1, 'boosting_type': 'gbdt', 'feature_fraction': 0.6, 'lambda_l1': 0.2, 'lambda_l2': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 19, 'metric': 'binary_error', 'min_data_in_leaf': 30, 'num_leaves': 43, 'objective': 'binary'}
Mejor score para Platform_index = 1 es: AUC = 0.714



Cross-validador temporal

In [17]:
from sklearn.metrics import roc_auc_score

In [79]:
mdl_lgb = lgb.LGBMClassifier(n_jobs=-1, task='refit')
tscv = TimeSeriesSplit(n_splits=9)
for i in [3, 2, 1]:
    print('Entrenando modelo para Platform_index = {}'.format(i))
    train_1 = train[train['Platform_index']==i]
#     train_1 = train_1.sort_values(by=['DateAvSigVersion_fulldifftotal'], ascending=False).fillna(-99)
    print("Nº de casos en el train = {}".format(train_1.shape[0]))
    X = train_1.loc[:, sel_cols]
    y = train_1.loc[:,'HasDetections']
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        lgb_model.fit(X_train, y_train)
        AUC = roc_auc_score(y_true=y_test,
                      y_score=lgb_model.predict(X_test))
        print('Mejor score para Platform_index = {} es: AUC = {}\n'.format(i, round(AUC, 3)))

Entrenando modelo para Platform_index = 3
Nº de casos en el train = 14371
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Mejor score para Platform_index = 3 es: AUC = 0.612

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.6s finished


Mejor score para Platform_index = 3 es: AUC = 0.661

Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Mejor score para Platform_index = 1 es: AUC = 0.652

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    5.1s finished


Mejor score para Platform_index = 1 es: AUC = 0.647

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    5.6s finished


Mejor score para Platform_index = 1 es: AUC = 0.651



In [89]:
lgb_model.cv_results_

{'mean_fit_time': array([0.85221068, 0.87714799, 1.12271261, 1.11535676, 0.86668309,
        0.89432247, 1.11534588, 1.10371947, 0.92180626, 0.90633496,
        1.12618677, 1.14261007, 1.0142471 , 1.00944066, 1.1247348 ,
        1.14086699, 0.64354897, 0.64703941, 0.77088205, 0.78073112,
        0.64652737, 0.63155897, 0.78355543, 0.78955634, 0.65919741,
        0.66965469, 0.77965562, 0.78662785, 0.72328432, 0.71400158,
        0.69906712, 0.67530608]),
 'std_fit_time': array([0.01674874, 0.03997479, 0.02007766, 0.0094622 , 0.00356062,
        0.01721581, 0.01317042, 0.00486124, 0.0169124 , 0.00901381,
        0.00740493, 0.03142621, 0.00402471, 0.00141628, 0.00749134,
        0.02656624, 0.01754608, 0.02039693, 0.00183282, 0.00440963,
        0.00844551, 0.01012898, 0.00421385, 0.02838774, 0.00403321,
        0.02067302, 0.00313479, 0.00223074, 0.02336763, 0.00653729,
        0.04749717, 0.02570644]),
 'mean_score_time': array([0.07871071, 0.08504987, 0.08939695, 0.08986378, 0.080940

In [49]:
mdl_lgb.feature_importances_

array([  0,  42,  14,  21, 258,  69,  29,   1,  95,  19,   8,  28,  80,
        17,   9,  11,   0,  26,  36,   8,   7,  26,  20,  16,   3,  59,
        62,  59,  18,  53,  71,  26,  25,  42,  11,  82,  38,  62,   0,
         0,   0,  17,  23,  17,  15,   4,  37,   7,   3,   3, 100,  99,
         0,   0,   1,  15,  12,   9,   9, 151,  12,   0,   8,   1,  20,
        10,   0,   0,  21,  32,  13, 119,  17,  30,  56,  14,   0,   0,
         0,   2,  65,  18,  15, 110, 125,   0, 107, 131,   0,   0,   0,
         0,   0,  29,  60,   6,   1,   5])

In [55]:
dict_importances = dict()
for i, j in zip(sel_cols, mdl_lgb.feature_importances_):
    dict_importances[i] = j

In [64]:
df_imp = pd.DataFrame({'col': sel_cols, 'imp': mdl_lgb.feature_importances_})

In [65]:
df_imp.sort_values('imp', ascending=False)

,col,imp
4,AVProductStatesIdentifier,258
59,SmartScreen_index,151
87,AvSigVersion_2,131
84,AppVersion_3,125
71,Census_OSInstallTypeName_index,119
83,AppVersion_2,110
86,AvSigVersion_1,107
50,Wdft_IsGamer,100
51,Wdft_RegionIdentifier,99
8,CountryIdentifier,95


In [73]:
col_sel_2 = df_imp.loc[:35, 'col']

In [74]:
col_sel_2

0                                                IsBeta
1                                      RtpStateBitfield
2                                      IsSxsPassiveMode
3                             DefaultBrowsersIdentifier
4                             AVProductStatesIdentifier
5                                   AVProductsInstalled
6                                     AVProductsEnabled
7                                                HasTpm
8                                     CountryIdentifier
9                                        CityIdentifier
10                               OrganizationIdentifier
11                                    GeoNameIdentifier
12                          LocaleEnglishNameIdentifier
13                                              OsBuild
14                                              OsSuite
15                                          IsProtected
16                                      AutoSampleOptIn
17                                              